#### Tuesday, November 5, 2024

mamba activate langchain-chroma

[Build a Local RAG Application](https://python.langchain.com/docs/tutorials/local_rag/)

This notebook uses Ollama to serve up Llama 3.1 8b model. It can also run without errors with Llama 3.2 3b.

This all runs in one pass.

In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

In [3]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [4]:
docs[0]

Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.')

In [5]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    # model="llama3.1:8b",
       model="llama3.1:latest",
       #model="llama3.2:latest",
)

In [6]:
response_message = model.invoke(
    "Simulate a rap battle between Stephen Colbert and John Oliver"
)

print(response_message.content)

Here's a simulated rap battle between Stephen Colbert and John Oliver:

**The Scene:** A packed comedy club, with the audience cheering and chanting for more. Stephen Colbert and John Oliver take the stage, each confident in their own rhyming abilities.

**Stephen Colbert (as Bored to Tears):**

Yo, I'm Bored to Tears, the king of satire's throne
My show's been a hit, since 2005, you know
Daily Show, that's my claim to fame
Making fun of idiots, that's just part of the game

You think you're funny, John Oliver, but you're whack
A British import, trying to take my spot back
Last Week Tonight, sounds like a snooze fest too
My audience is bigger, boo!

**John Oliver (as The Truth Teller):**

Hold up, Bored, let me set the record straight
I'm The Truth Teller, here to bring the facts to the plate
Your show's all jokes and no substance, it's true
But I'm over here, exposing the lies that you're too afraid to pursue

My show may not be as funny, but it's informative gold
Exposing the scams, 

#### Using in a chain

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)

'The main themes in these documents are:\n\n1. **Task Decomposition**: Breaking down complex tasks into smaller, manageable subgoals to facilitate efficient completion.\n2. **Autonomous Agent System**: A system powered by Large Language Models (LLMs) that can perform planning, task execution, and reflection/refinement on complex tasks.\n3. **Planning and Organization**: The importance of planning ahead and organizing tasks to ensure efficient handling of complex tasks.\n\nThese themes are related to the design of a LLM-powered autonomous agent system, which aims to facilitate the completion of complex tasks through decomposition, planning, and self-reflection.'

#### Q&A

In [8]:
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

# Run
chain.invoke({"context": docs, "question": question})

'There are three approaches to task decomposition: (1) using LLM with simple prompting, (2) employing task-specific instructions, and (3) incorporating human inputs. These methods enable breaking down complex tasks into manageable subgoals, facilitating efficient handling and planning. This process is crucial for agent systems powered by Large Language Models (LLM).'

#### Q&A with retrieval

In [9]:
retriever = vectorstore.as_retriever()

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [10]:
question = "What are the approaches to Task Decomposition?"

qa_chain.invoke(question)

'Task decomposition can be done through three approaches: (1) using Large Language Models (LLM) with simple prompting, (2) employing task-specific instructions, or (3) incorporating human inputs. These methods enable agents to break down complex tasks into smaller, manageable subgoals for efficient handling. This approach is part of the planning component in a LLM-powered autonomous agent system.'